In [1]:
import nbodykit as nkit
import numpy as npy
from nbodykit.source.catalog import CSVCatalog
from nbodykit.source.catalog import ArrayCatalog
from matplotlib import pyplot
import healpy as hp
import pmesh
import time
import sys;
from importlib import reload 
import scipy as scpy
sys.path.append("/home/chris/Dropbox/Document/Metric reconstruction/")

In [2]:
# Importing lcmetric modules
import lcmetric
import lcmetric.lightcone as lightcone_it_mg_lm
import lcmetric.utils as ut
import lcmetric.clightcone_CIC as lc_CIC
import lcmetric.cutils as utC

In [10]:
cone_dtype = npy.double # Data type for light-cone
NR = 430 # Radial resolution. NOTE: the actual size of any light-cone mesh field is NR+2 
NSIDE = 80 #Healpix nside
NPIX = 12 * NSIDE * NSIDE #Number of pixels

# In Mpc, to convert from Mpc h^-1, multiply cosmo_paras['h'] / L_unit
# H_0 is cosmo_paras['h'] * 100 with L_unit = 3e5
L_unit = 3e5 

N_snap = 256 #snapshot mesh resolution
N_samples = 256 #snapshot number of particles

origin = npy.array([0, 0, 0.0], dtype=cone_dtype) #light-cone origin


In [4]:
# Setting initial cosmic parameters
cosmo_paras={'h':0.69, 'Omega_m':0.31, 'Omega_L':0.69}
init_zel_z = 9 #Redshift where n-body simulation starts
init_z = 0.3 
init_a = 1/(1+init_z)
final_z = 0.03
final_a = 1/(1+final_z)

# 1/H(z)
def Hint(z, cosmo_paras):
    return 1/(cosmo_paras['h']*100*npy.sqrt(cosmo_paras['Omega_m']*(1+z)**3 + cosmo_paras['Omega_L']))

# H(z)
def H(z, cosmo_paras):
    return (cosmo_paras['h']*100/(1+z)*npy.sqrt(cosmo_paras['Omega_m']*(1+z)**3 + cosmo_paras['Omega_L']))

# integrate 1/H(z) - r, to inversely solve z for given r
def inverse_Hint(z, r, cosmo_paras):
    return npy.abs(scpy.integrate.quad(Hint, 0, z,args=(cosmo_paras))[0] - r)

# Initial comoving distance
init_r = scpy.integrate.quad(Hint, 0, init_z,args=(cosmo_paras))[0]

# Final comoving distance
final_r = scpy.integrate.quad(Hint, 0, final_z,args=(cosmo_paras))[0]

# Initial Hubble
Hi = H(init_z, cosmo_paras)

#Final Hubble
Hf = H(final_z, cosmo_paras)
H0 = cosmo_paras['h'] * 100
L_snap = 512 / cosmo_paras['h'] / L_unit

In [5]:
# Locating radial distance for radial mesh n
def to_r(n,nr = NR):
    return n / nr * (init_r - final_r) + final_r

In [6]:

def init(NR, N_snap, NSIDE, cone, snap_i, snap_f, snap_zel=None):
    """Sample function of setting-up initial data
    Parameters
    ----------
    NR : int
        Radial resolution. NOTE: the actual size of any light-cone mesh field is NR+2 
    NSIDE: init
        healpix NSIDE
    cone: str
        Leading filename with path of the cone file
    snap_i: str
        Leading filename with path of the intial snap
    snap_f: str
        Leading filename with path of the final snap
    snap_f: str, optional
        Leading filename with path of the snapshots as the initial conditon for N-body
    Returns
    -------
    delta: float, shape(NR+2, NPIX)
    vw: float, shape(NR+2, NPIX)
    counts: float, shape(NR+2, NPIX)
    Phi_i: float, shape(NPIX,)
    Pi_i: float, shape(NPIX,)
    Phi_f: float, shape(NPIX,)
    void
    """
    ## Start reading initial snapshot and setting initial data
    f = CSVCatalog( \
        snap_i+'*',['x','y','z','vx','vy','vz','phi'])
    f['Position'] = (f['x'][:, None]) * [1/cosmo_paras['h']/L_unit, 0, 0] + (f['y'][:, None]) * [0, 1/cosmo_paras['h']/L_unit, 0] + (f['z'][:, None]) * [0, 0, 1/cosmo_paras['h']/L_unit]

    f.attrs['BoxSize'] = L_snap 

    rf = (f.to_mesh(N_snap).to_real_field(normalize=False) * ( N_snap**3 / f['Position'].shape[0] )  - 1.0 ) \
    *(1.5 * (cosmo_paras['h']*100)**2 * cosmo_paras['Omega_m']/init_a )
                                                       
    f = CSVCatalog(snap_f+'*',['x','y','z','vx','vy','vz','phi'])
    f['Position'] = (f['x'][:, None]) * [1/cosmo_paras['h']/L_unit, 0, 0] + (f['y'][:, None]) * [0, 1/cosmo_paras['h']/L_unit, 0] + (f['z'][:, None]) * [0, 0, 1/cosmo_paras['h']/L_unit]
    f.attrs['BoxSize'] = L_snap 

    rf_f = (f.to_mesh(N_snap).to_real_field(normalize=False) * ( N_snap**3 / f['Position'].shape[0] )  - 1.0 ) \
        *(1.5 * (cosmo_paras['h']*100)**2 * cosmo_paras['Omega_m']/final_a )

    if(snap_zel != None):
        f = CSVCatalog(snap_zel+'*',['x','y','z','vx','vy','vz','phi'])
        f['Position'] = (f['x'][:, None]) * [1/cosmo_paras['h']/L_unit, 0, 0] + (f['y'][:, None]) * [0, 1/cosmo_paras['h']/L_unit, 0] + (f['z'][:, None]) * [0, 0, 1/cosmo_paras['h']/L_unit]

        f.attrs['BoxSize'] = L_snap 

        rf_i = (f.to_mesh(N_snap).to_real_field(normalize=False) * ( N_snap**3 / f['Position'].shape[0] )  - 1.0 ) \
            *(1.5 * (cosmo_paras['h']*100)**2 * cosmo_paras['Omega_m'] * (1+init_zel_z) )
        
        # Calculating relativistical corrections for particle position
        rf_i0 = rf_i.copy()
        rf_i1 = rf_i.copy()
        rf_i2 = rf_i.copy()
        Phi_snap_I = 5*ut.inverse_Lap(rf_i,L_snap,N_snap)
        rf_i0.value = ut.inverse_derv(Phi_snap_I, L_snap,N_snap, 0)
        rf_i1.value = ut.inverse_derv(Phi_snap_I, L_snap,N_snap, 1)
        rf_i2.value = ut.inverse_derv(Phi_snap_I, L_snap,N_snap, 2)
    
    ## Start reading initial snapshot and setting initial data
    f = CSVCatalog(cone+'*', \
               ['x','y','z','vx','vy','vz','phi'],dtype='f8')
    
    f['Particles'] = (f['x'][:, None]) * npy.array([1/cosmo_paras['h']/L_unit, 0, 0, 0, 0, 0] , dtype = cone_dtype)\
    + (f['y'][:, None]) * npy.array([0, 1/cosmo_paras['h']/L_unit, 0, 0, 0, 0], dtype = cone_dtype) \
    + (f['z'][:, None]) * npy.array([0, 0, 1/cosmo_paras['h']/L_unit, 0, 0, 0], dtype = cone_dtype)\
    + (f['vx'][:, None]) * npy.array([0, 0, 0, 1/3e5, 0, 0], dtype = cone_dtype) \
    + (f['vy'][:, None]) * npy.array([0, 0, 0, 0, 1/3e5, 0], dtype = cone_dtype) \
    + (f['vz'][:, None]) * npy.array([0, 0, 0, 0, 0, 1/3e5], dtype = cone_dtype) 

    f.attrs['BoxSize'] = L_snap 
    f['Particles']=f['Particles'].rechunk(20000000)
    
    # Creating healpy coordinate list in cartesian coordinate
    theta_list, phi_list = hp.pix2ang(NSIDE, range(12*NSIDE**2))
    x_list = npy.array([ \
                             init_r * npy.sin(theta_list) * npy.cos(phi_list) + origin[0],        \
                             init_r * npy.sin(theta_list) * npy.sin(phi_list) + origin[1],        \
                             init_r * npy.cos(theta_list) + origin[2]                             \
        ])
    x_list = npy.ascontiguousarray(npy.transpose(x_list))

    xf_list = npy.array([ \
                             final_r * npy.sin(theta_list) * npy.cos(phi_list) + origin[0],        \
                             final_r * npy.sin(theta_list) * npy.sin(phi_list) + origin[1],        \
                             final_r * npy.cos(theta_list) + origin[2]                             \
        ])
    xf_list = npy.ascontiguousarray(npy.transpose(xf_list))
    
    
    # Setting initial Phi
    Phi_snap=rf.copy()
    Phi_snap.value = ut.inverse_Lap(rf,L_snap,N_snap)
    Phi = ut.interp(Phi_snap, Phi_snap.BoxSize/Phi_snap.Nmesh, x_list)

    # Setting initial Pi
    Pi = ut.f_r_derv(Phi_snap, Phi_snap.BoxSize/Phi_snap.Nmesh, origin, init_r, x_list)

    
    
    # Setting final Phi
    Phi_f_snap=rf_f.copy()
    Phi_f_snap.value = ut.inverse_Lap(rf_f,L_snap,N_snap)
    Phi_f = ut.interp(Phi_f_snap, Phi_f_snap.BoxSize/Phi_f_snap.Nmesh, xf_list)

    
 
        
    delta = npy.zeros((NR+2,NPIX),dtype= cone_dtype)
    delta.fill(-1)
    vw = npy.zeros((NR+2, NPIX),dtype= cone_dtype)
    counts = npy.zeros((NR+2, NPIX),dtype= cone_dtype)
    count_density = 1 / (L_snap / N_snap)**3 / (N_snap / N_snap)**3
    max_r = init_r
    min_r = final_r

    
    for d in f['Particles'].partitions:
        pdata = d.compute()
        if(snap_zel != None):
            pdata += rf_i0.readout(pdata[:,0:3])[:,None] * [1,0,0,0,0,0] \
            + rf_i1.readout(pdata[:,0:3])[:,None] * [0,1,0,0,0,0] \
            + rf_i2.readout(pdata[:,0:3])[:,None] * [0,0,1,0,0,0]
        lc_CIC.deposit(pdata, origin, delta, count_density, vw, counts, max_r, min_r, NR, NSIDE,0)
    
    vw /= counts
    vw=npy.nan_to_num(vw)


    return (delta,vw,counts,Phi,Pi,Phi_f)

In [ ]:
snap_i = \
'/media/chris/3b7ae93c-9459-4858-9b27-3209d1805b9a/draft_data/Metric_recon/cone_test_256/cone_test_256_z0p300.'
snap_f = \
'/media/chris/3b7ae93c-9459-4858-9b27-3209d1805b9a/draft_data/Metric_recon/cone_test_256/cone_test_256_z0p030.'
cone = \
'/media/chris/3b7ae93c-9459-4858-9b27-3209d1805b9a/draft_data/Metric_recon/cone_test_256/cone_test_256_lightcone.'
snap_zel = \
'/media/chris/3b7ae93c-9459-4858-9b27-3209d1805b9a/draft_data/Metric_recon/cone_test_256/cone_test_256_z9p000.'


delta,vw,counts,Phi,Pi,Phi_f = init(430, 256, NSIDE, cone, snap_i, snap_f, snap_zel)



In [ ]:
lc = lightcone_it_mg_lm.Lightcone(NSIDE, epsilon = 1e-12,grid='healpy',  alm_iter = 50,
                    depth = 5, n_vcycles = 100, npre = 16, npost = 16, lmax = 2*NSIDE-1, verbose=False)
lc.init_from_slice(init_z, init_r, delta, vw, Phi, Pi, 
                       cosmo_paras, final_z, final_r, Phi_f)
lc.build_lcmetric()